## Lecture 3 demo

There are 3 key themes to this lecture:

1. dates & times

2. strings

3. factors

First, let's load the packages we need:

In [3]:
library(tidyverse)
library(lubridate)
library(palmerpenguins)

*Note: if you have to install an R package that exists on CRAN, the command is: `install.packages("PACKAGE_NAME")`.*

And then let's limit the output of data frames in Jupyter to 6 lines:

In [4]:
options(repr.matrix.max.rows = 6)

## 1. Dates & times

We learned about how to make date and time objects, but why are we doing this? What kinds of things can we do with these that are more difficult if we left these as characters or numeric vectors?

- We can use date operations like `month`,`year` etc. 
- We can ask whether a date falls within a certain interval. Use the `interval` function to create an interval.
- It impacts modeling and data visualization.(you will see in DSCI 531)

Show them the datatypes image

In [5]:
datev <- "31-Jan-2011"
typeof(datev)
# Won't run
month(datev)

[1] "character"

ERROR: Error in as.POSIXlt.character(x, tz = tz(x)): character string is not in a standard unambiguous format


In [6]:
datev <- dmy("31-Jan-2011")
datev <- mdy("Jan-31-2011")
class(datev)
month(datev)

[1] "Date"

[1] 1

In [7]:
# setting up an interval 
my_interval <- interval(start = dmy("1-1-2019"),end = dmy("31-1-2019"))

- `%within%` is useful for pairing with `filter` to get rows in an interval

In [8]:
## see if 15-1-2019 is in this interval
dmy("15-1-2019") %within% my_interval

[1] TRUE

In [9]:
## How you apply this to a data frame
time_df <- tibble(date_col = c("15-1-2020","15-1-2019","20-1-2019","30-1-2019"),
                 other_col = c("some details1","details2","details3","details4"))
time_df
time_df |> mutate(date_col = dmy(date_col)) |> filter(date_col %within% my_interval) 

date_col,other_col
<chr>,<chr>
15-1-2020,some details1
15-1-2019,details2
20-1-2019,details3
30-1-2019,details4


date_col,other_col
<date>,<chr>
2019-01-15,details2
2019-01-20,details3
2019-01-30,details4


You will do this in the worksheet using `lubridate`'s `interval` function to create the interval, and then `filter` + `%within%` to subset rows from that interval.

### Clicker 1

Show them cheat sheet. https://www.rstudio.com/resources/cheatsheets/

## 2. Strings

My most common use for strings manipulation in data wrangling is filtering for rows where I have partial matches, and then replacing strings. The former can be done by pairing `filter` with`stringr`'s `str_detect` function, while the latter can be done by pairing `mutate` with `stringr`'s `str_replace` function.

Other functions that can be quite useful are if you want to split a column in a dataframe into 2 separate columns or vice versa. The former can be done using `separate`, while the latter can be done using `unite`.

Below I will demonstrate both on the `lakers` data frame below (which is the Los Angeles Lakers 2008-2009 basketball data set from the `lubridate` package):

In [10]:
lakers

date,opponent,game_type,time,period,etype,team,player,result,points,type,x,y
<int>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<int>
20081028,POR,home,12:00,1,jump ball,OFF,,,0,,NA,NA
20081028,POR,home,11:39,1,shot,LAL,Pau Gasol,missed,0,hook,23,13
20081028,POR,home,11:37,1,rebound,LAL,Vladimir Radmanovic,,0,off,NA,NA
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
20090414,UTA,home,00:27,4,turnover,LAL,Andrew Bynum,,0,,NA,NA
20090414,UTA,home,00:21,4,shot,UTA,Kyle Korver,missed,0,3pt,41,25
20090414,UTA,home,00:20,4,rebound,LAL,Luke Walton,,0,def,NA,NA


Let's find all the rows where the player has the name Kyle:

In [11]:
lakers |>
  filter(str_detect(player, "Kyle"))

date,opponent,game_type,time,period,etype,team,player,result,points,type,x,y
<int>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<int>
20081222,MEM,away,09:33,1,shot,MEM,Kyle Lowry,missed,0,3pt,13,28
20081222,MEM,away,04:59,1,rebound,MEM,Kyle Lowry,,0,def,NA,NA
20081222,MEM,away,04:39,1,shot,MEM,Kyle Lowry,missed,0,jump bank,30,12
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
20090414,UTA,home,00:42,4,free throw,UTA,Kyle Korver,made,1,,NA,NA
20090414,UTA,home,00:42,4,free throw,UTA,Kyle Korver,made,1,,NA,NA
20090414,UTA,home,00:21,4,shot,UTA,Kyle Korver,missed,0,3pt,41,25


Wait !!! I am interested in knowing about Trevor as well.

In [12]:
# Filter that contain pattern "Kyle" or "Trevor" in column player. 
# We can do this by creating a pattern which would match them "Kyle|Trevor"
lakers |> 
    filter(str_detect(player,"Kyle|Trevor"))

date,opponent,game_type,time,period,etype,team,player,result,points,type,x,y
<int>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<int>
20081028,POR,home,02:31,1,rebound,LAL,Trevor Ariza,,0,def,NA,NA
20081028,POR,home,02:11,1,shot,LAL,Trevor Ariza,made,2,dunk,25,6
20081028,POR,home,01:31,1,shot,LAL,Trevor Ariza,made,3,3pt,46,17
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
20090414,UTA,home,00:42,4,free throw,UTA,Kyle Korver,made,1,,NA,NA
20090414,UTA,home,00:42,4,free throw,UTA,Kyle Korver,made,1,,NA,NA
20090414,UTA,home,00:21,4,shot,UTA,Kyle Korver,missed,0,3pt,41,25


What if I am interested in knowing all players that starts with Ky ?

In [13]:
## That starts with "Ky".
lakers |> 
    filter(str_detect(player,"^Ky"))

date,opponent,game_type,time,period,etype,team,player,result,points,type,x,y
<int>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<int>
20081222,MEM,away,09:33,1,shot,MEM,Kyle Lowry,missed,0,3pt,13,28
20081222,MEM,away,04:59,1,rebound,MEM,Kyle Lowry,,0,def,NA,NA
20081222,MEM,away,04:39,1,shot,MEM,Kyle Lowry,missed,0,jump bank,30,12
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
20090414,UTA,home,00:42,4,free throw,UTA,Kyle Korver,made,1,,NA,NA
20090414,UTA,home,00:42,4,free throw,UTA,Kyle Korver,made,1,,NA,NA
20090414,UTA,home,00:21,4,shot,UTA,Kyle Korver,missed,0,3pt,41,25


Sorry, but now I am interested in knowing all the player that ends with wry

In [14]:
## That ends with "wry"
lakers |> 
    filter(str_detect(player,"wry$"))

date,opponent,game_type,time,period,etype,team,player,result,points,type,x,y
<int>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<int>
20081222,MEM,away,09:33,1,shot,MEM,Kyle Lowry,missed,0,3pt,13,28
20081222,MEM,away,04:59,1,rebound,MEM,Kyle Lowry,,0,def,NA,NA
20081222,MEM,away,04:39,1,shot,MEM,Kyle Lowry,missed,0,jump bank,30,12
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
20090403,HOU,home,11:38,4,foul,HOU,Kyle Lowry,,0,personal,NA,NA
20090403,HOU,home,11:25,4,rebound,HOU,Kyle Lowry,,0,def,NA,NA
20090403,HOU,home,10:45,4,shot,HOU,Kyle Lowry,missed,0,layup,25,6


Do we have any rows with no player names ? Now, let's filter for all the rows that don't have a player name 

In [15]:
## That contain empty value in column player $^
lakers |> 
    filter(str_detect(player,"^$"))

date,opponent,game_type,time,period,etype,team,player,result,points,type,x,y
<int>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<int>
20081028,POR,home,12:00,1,jump ball,OFF,,,0,,NA,NA
20081028,POR,home,07:49,1,timeout,POR,,,0,regular,NA,NA
20081028,POR,home,07:24,1,sub,POR,,,0,,NA,NA
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
20090414,UTA,home,02:46,4,sub,UTA,,,0,,NA,NA
20090414,UTA,home,02:46,4,sub,UTA,,,0,,NA,NA
20090414,UTA,home,02:06,4,rebound,UTA,,,0,,NA,NA


Next, let's change some of the text so it is more readable - so "3pt" to "3 point":

In [16]:
lakers |>
  mutate(type = str_replace(type, "3pt", "3 point"))

date,opponent,game_type,time,period,etype,team,player,result,points,type,x,y
<int>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<int>
20081028,POR,home,12:00,1,jump ball,OFF,,,0,,NA,NA
20081028,POR,home,11:39,1,shot,LAL,Pau Gasol,missed,0,hook,23,13
20081028,POR,home,11:37,1,rebound,LAL,Vladimir Radmanovic,,0,off,NA,NA
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
20090414,UTA,home,00:27,4,turnover,LAL,Andrew Bynum,,0,,NA,NA
20090414,UTA,home,00:21,4,shot,UTA,Kyle Korver,missed,0,3 point,41,25
20090414,UTA,home,00:20,4,rebound,LAL,Luke Walton,,0,def,NA,NA


Ohhh.. What if I want to replace all the players with missing value having some other default value in there ?

In [17]:
## That contain empty value in column player, replace it with a name
lakers |> 
    mutate(player = str_replace(player,"$^","no name"))

date,opponent,game_type,time,period,etype,team,player,result,points,type,x,y
<int>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<int>
20081028,POR,home,12:00,1,jump ball,OFF,no name,,0,,NA,NA
20081028,POR,home,11:39,1,shot,LAL,Pau Gasol,missed,0,hook,23,13
20081028,POR,home,11:37,1,rebound,LAL,Vladimir Radmanovic,,0,off,NA,NA
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
20090414,UTA,home,00:27,4,turnover,LAL,Andrew Bynum,,0,,NA,NA
20090414,UTA,home,00:21,4,shot,UTA,Kyle Korver,missed,0,3pt,41,25
20090414,UTA,home,00:20,4,rebound,LAL,Luke Walton,,0,def,NA,NA


Now let's try to separate out column `player` into 2 columns `player_f_name` and `player_l_name`. 

In [18]:
new_lakers <- lakers |> separate(player,into = c("player_f_name","player_l_name"),sep = " ")
head(new_lakers)

Warning message:
“Expected 2 pieces. Additional pieces discarded in 59 rows [2417, 2419, 5601, 5615, 5617, 5655, 5666, 5668, 5681, 5684, 5704, 5714, 5718, 5777, 5788, 7745, 7748, 7770, 7785, 7816, ...].”
Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 5398 rows [1, 35, 40, 44, 50, 51, 52, 69, 70, 81, 82, 83, 84, 96, 97, 105, 110, 114, 123, 127, ...].”


,date,opponent,game_type,time,period,etype,team,player_f_name,player_l_name,result,points,type,x,y
,<int>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<int>
1,20081028,POR,home,12:00,1,jump ball,OFF,,NA,,0,,NA,NA
2,20081028,POR,home,11:39,1,shot,LAL,Pau,Gasol,missed,0,hook,23,13
3,20081028,POR,home,11:37,1,rebound,LAL,Vladimir,Radmanovic,,0,off,NA,NA
4,20081028,POR,home,11:25,1,shot,LAL,Derek,Fisher,missed,0,layup,25,6
5,20081028,POR,home,11:23,1,rebound,LAL,Pau,Gasol,,0,off,NA,NA
6,20081028,POR,home,11:22,1,shot,LAL,Pau,Gasol,made,2,hook,25,10


Now let's those columns back to a single column `player`.

In [19]:
new_lakers |> unite(col = "player", c(player_f_name,player_l_name),sep= " ")

date,opponent,game_type,time,period,etype,team,player,result,points,type,x,y
<int>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<int>
20081028,POR,home,12:00,1,jump ball,OFF,NA,,0,,NA,NA
20081028,POR,home,11:39,1,shot,LAL,Pau Gasol,missed,0,hook,23,13
20081028,POR,home,11:37,1,rebound,LAL,Vladimir Radmanovic,,0,off,NA,NA
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
20090414,UTA,home,00:27,4,turnover,LAL,Andrew Bynum,,0,,NA,NA
20090414,UTA,home,00:21,4,shot,UTA,Kyle Korver,missed,0,3pt,41,25
20090414,UTA,home,00:20,4,rebound,LAL,Luke Walton,,0,def,NA,NA


### Clicker 2

## 3. Factors

Show them datatype image....

Factors are a kind of vector in R. And remember - data frame columns are usually made up of vectors - so you most commonly encounter them as columns in a data frame.

You can think factor == categoricals (Pandas in Python has this data type too and uses the name categorical). They are very useful for modeling and data visualization.

Let's see what happens when we convert the `etype` column in the `lakers` data frame to a factor, as well as how we can do that:

First, let's look at what the column originally is:

In [20]:
class(lakers$etype)
str(lakers$etype)
# str(lakers)

[1] "character"

 chr [1:34624] "jump ball" "shot" "rebound" "shot" "rebound" "shot" "foul" ...


In [21]:
myname <- "tiff"
str(myname)

 chr "tiff"


Then we can use `as.factor` to convert it to a factor column:

In [22]:
lakers_factor <- lakers |>
  mutate(etype = as.factor(etype))

str(lakers_factor$etype)
levels(lakers_factor$etype)

 Factor w/ 10 levels "ejection","foul",..: 4 6 5 6 5 6 2 3 2 6 ...


[1] "ejection"   "foul"       "free throw" "jump ball"  "rebound"   
 [6] "shot"       "sub"        "timeout"    "turnover"   "violation"

You can use `fct_reorder` to order one variable by another. The factor is the grouping variable and the default summarizing function is median but you can specify something else.

In [23]:
lakers_factor <- lakers_factor |> mutate(etype = fct_reorder(etype,period,min))

In [24]:
lakers_factor$etype |> levels()

[1] "foul"       "free throw" "jump ball"  "rebound"    "shot"      
 [6] "sub"        "timeout"    "turnover"   "violation"  "ejection"

In [25]:
lakers_factor

date,opponent,game_type,time,period,etype,team,player,result,points,type,x,y
<int>,<chr>,<chr>,<chr>,<int>,<fct>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<int>
20081028,POR,home,12:00,1,jump ball,OFF,,,0,,NA,NA
20081028,POR,home,11:39,1,shot,LAL,Pau Gasol,missed,0,hook,23,13
20081028,POR,home,11:37,1,rebound,LAL,Vladimir Radmanovic,,0,off,NA,NA
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
20090414,UTA,home,00:27,4,turnover,LAL,Andrew Bynum,,0,,NA,NA
20090414,UTA,home,00:21,4,shot,UTA,Kyle Korver,missed,0,3pt,41,25
20090414,UTA,home,00:20,4,rebound,LAL,Luke Walton,,0,def,NA,NA


In the lab, you will explore how factor levels impact data visualization. And next block, in DSCI 552 you will start to use them for statistical analysis.

In [26]:
c(1,"hello","yue")

[1] "1"     "hello" "yue"

In [27]:
list(1,"hello","yue")

[[1]]
[1] 1

[[2]]
[1] "hello"

[[3]]
[1] "yue"